In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output
import re, time, random, requests, platform, subprocess, os
from string import ascii_letters, whitespace
import unicodedata
from IPython.display import clear_output

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15'}
FQDN = 'https://s*****.**'
responce = requests.get(FQDN)

print("request denied") if responce.status_code != 200 else print("ok")

In [ ]:
def get_html(url, params=None):
    r = requests.get(url, headers=header, params=params)
    return r

def open_file(path):
    if platform.system() == "Windows":
        os.startfile(path)
    elif platform.system() == "Darwin":
        subprocess.Popen(["open", path])
    else:
        subprocess.Popen(["xdg-open", path])

def to_eng(text):
    return (' '.join(re.findall(r'[A-z0-9.№]+', text))).replace('. ','').strip('.').strip()

def to_rus(text):
    return (' '.join(re.findall(r'[А-я0-9.№]+', text))).replace('. ','').strip('.').strip()

def clean(text):
    if any(letter.isalpha() for letter in text):
        return text
    else:
        return ''
            
def post_proc(text: str):
    text_eng = (' '.join(re.findall(r'[A-z0-9\'äáàåãâčçÇČèéěÉğíóòöōôÓşśšŠúüÜýŽñßəıøł@&’.№]+', text)))
    text_rus = (' '.join(re.findall(r'[А-я0-9ё.«»№]+', text)))
    if any(letter.isalpha() for letter in text_eng):
        pass
    else:
        text_eng = ''
        
    if any(letter.isalpha() for letter in text_rus):
        pass
    else:
        text_rus = ''

    text_eng = list(text_eng.split())
    text_rus = list(text_rus.split())
    
    unique_words_eng = []
    dupl_words_eng = []
    unique_words_rus = []
    dupl_words_rus = []
    
    for w in text_eng:
        if w not in unique_words_eng:
            unique_words_eng.append(w)
        else:
            if any(letter.isalpha() for letter in w):
                dupl_words_eng.append(w)
        
    for w in text_rus:
        if w not in unique_words_rus:
            unique_words_rus.append(w)
    unique_words_rus.extend(dupl_words_eng)
    
    clean_eng = ' '.join(unique_words_eng).strip().lstrip('.').strip().replace(' . ',' ')
    clean_rus = ' '.join(unique_words_rus).strip().lstrip('.').strip().replace(' . ',' ')
    return clean_eng, clean_rus

In [ ]:
def breeding_page(p_url:str):
    '''
    Web scraping breeding page
    Example p_url: https://s****.**/catalog/type-4/?page=97
    '''
    response = get_html(p_url)
    soup = BeautifulSoup(response.text, 'lxml')
    # название объекта
    tittle_object = [e.text.strip() for e in soup.find_all('a', class_ = 'camp__title')]
    # статус работы
    status_object = [e.find('span', style = True).text if e.find('span', style = True) else 'Открыто' for e in soup.find_all('div', class_ = 'camp-min__head-name-inner')]
    # ссылка на объект
    url_object = [FQDN + e.get('href') for e in soup.find_all('a', class_ = 'camp__title')]
    # страна объекта
    country_object = [e.text.strip().split('\n')[0] for e in soup.find_all('div', class_ = 'location')]
    # город объекта
    city_object = [e.text.strip().split('\n')[-1] for e in soup.find_all('div', class_ = 'location')]
    # рейтинг объекта
    raiting_object = [e.text.strip() for e in soup.find_all('div', class_ = 'evaluation__rating')]
    # количество отзывов на объект
    response_object = [re.sub('\D', '', e.text.strip().partition('отзывов')[0]) 
                       if 'отзывов' in e.text.strip() else 0 
                       for e in soup.find_all('div', class_ = 'camp-min__head-rating')]
    # cтоимость обучения
    price_object = [e.text.strip().partition('от ')[-1].partition('Подать документы')[0].replace(' ','').replace('\n','') 
                    if 'от ' in e.text.strip() else 'Нет данных' 
                    for e in soup.find_all('div', class_ = 'camp-min__head-rating')]
    
    
    assert(len(tittle_object)==len(status_object)==len(url_object)==len(country_object)==len(city_object)==len(raiting_object)==len(response_object)==len(price_object)), 'Assert Error'
    time.sleep(random.uniform(.6, 1.5))
    
    return tittle_object,status_object,url_object,country_object,city_object,raiting_object,response_object,price_object
    

In [ ]:
# Главная
FQDN = 'https://s****.**'
response = get_html(FQDN)
soup = BeautifulSoup(response.text, 'lxml')
class_object = [e.text.strip() for e in soup.find_all('div', class_ = 'sect-institutions__title')]
class_url = [[FQDN + e1.get('data-href') for e1 in e.find_all(attrs={'data-href':True})] for e in soup.find_all(attrs={"class": "sect-institutions__box"})][0]

In [ ]:
df = pd.DataFrame()

for class_object_tag, class_url_tag in zip(class_object[:], class_url[:]):
    response = get_html(class_url_tag)
    soup = BeautifulSoup(response.text, 'lxml')
    last_page = int([e.text for e in soup.find_all('a', class_ = 'pagination__link')][-1])
    
    data = pd.DataFrame()
    type_object = []
    tittle_object = []
    status_object = []
    url_object = []
    country_object = []
    city_object = []
    raiting_object = []
    response_object = []
    price_object = []

    for num_page in range(last_page):
        elem1, elem2, elem3, elem4, elem5, elem6, elem7,elem8 = breeding_page(class_url_tag +'?page=' + str(num_page+1))
        tittle_object.extend(elem1)
        status_object.extend(elem2)
        url_object.extend(elem3)
        country_object.extend(elem4)
        city_object.extend(elem5)
        raiting_object.extend(elem6)
        response_object.extend(elem7)
        price_object.extend(elem8)

        clear_output(wait=True)
        print(class_url_tag +'?page=' + str(num_page+1))
        print(len(elem1),len(elem2),len(elem3),len(elem4),len(elem5),len(elem6),len(elem7),len(elem8))
    
    type_object = [class_object_tag]*len(tittle_object)
    print(len(tittle_object),len(status_object),len(url_object),len(country_object),len(city_object),len(raiting_object),len(response_object),len(price_object),len(type_object))
    
    data['Тип объекта'] = type_object
    data['Полное название'] = tittle_object
    data['Статус'] = status_object
    data['Ссылка'] = url_object
    data['Страна'] = country_object
    data['Город'] = city_object
    data['Оценка'] = raiting_object
    data['Количество отзывов'] = response_object
    data['Минимальная стоимость'] = price_object
    data['Оценка'] = data['Оценка'].astype('float')
    data['Количество отзывов'] = data['Количество отзывов'].astype('int32')
    df = pd.concat([df, data], ignore_index=True)

In [ ]:
# пост-процессинг
# распарсим полные названия на eng/rus
df['Полное название(eng)'] = df['Полное название'].apply(lambda x: post_proc(x)[0] if isinstance(x, str) else x)
df['Полное название(rus)'] = df['Полное название'].apply(lambda x: post_proc(x)[1] if isinstance(x, str) else x)
   
# распарсим данные по стоимости,валюте,периоде
df['Cтоимость(в цифрах)'] = df['Минимальная стоимость'].apply(lambda x: re.sub('\D', '', x.split('/')[0].replace('.00','')) if x else None)
df['Cтоимость(в цифрах)'] = pd.to_numeric(df['Cтоимость(в цифрах)'], downcast='signed', errors='coerce')
df['Cтоимость(в цифрах)'] = df['Cтоимость(в цифрах)'].astype('Int64')
df['Валюта'] = df['Минимальная стоимость'].apply(lambda x: x.split('/')[0].partition('.00')[-1] if '/' in x and '.00' in x else x)
df['Период'] = df['Минимальная стоимость'].apply(lambda x: x.split('/')[-1] if '/' in x else x)

df = df.reindex(columns=['Тип объекта','Полное название','Полное название(eng)','Полное название(rus)',
                          'Статус','Ссылка','Страна','Город','Оценка','Количество отзывов',
                          'Минимальная стоимость','Cтоимость(в цифрах)','Валюта','Период'])

In [ ]:
foundation = []
age = []
learning_type = []
n_students = []
programs = []
placement = []
rating = []
advantage = []
temp_count_n_stud = []
temp_foundation = []
count=0
error = {}

for p_url in df['Ссылка'][:].iteritems():
    clear_output(wait=True)
    print(p_url[1])
    response = get_html(p_url[1])
    soup = BeautifulSoup(response.text, 'lxml')
    count_f = count_age = count_l = count_n_stud = count_prog = count_place = count_rait = count_adv = 0
    for e in soup.find_all('div',class_ = 'accordion__box'):
        for e1 in e.find_all('li'):
            if 'основан' in e1.text.strip().lower() and ':' in e1.text.strip():
                count_f += 1
                temp_foundation.append(e1.text.strip().split(':')[-1].strip())
            if 'Возраст' in e1.text.strip() and (':' in e1.text.strip() or '–' in e1.text.strip()):
                count_age += 1
                age.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
            if ('Вид обучения' in e1.text.strip() 
                or 'Формы обучения' in e1.text.strip()) and ':' in e1.text.strip():
                count_l += 1
                learning_type.append(e1.text.strip().split(':')[-1].strip())
            if 'количество' in e1.text.strip().lower() and ':' in e1.text.strip():
                count_n_stud += 1
                temp_count_n_stud.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
            if 'Программы' in e1.text.strip() and ':' in e1.text.strip():
                count_prog += 1
                programs.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
            if ('Размещение' in e1.text.strip() 
                or 'Тип проживания' in e1.text.strip()) and ':' in e1.text.strip():
                count_place += 1
                placement.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
            if 'Рейтинг' in e1.text.strip() and ':' in e1.text.strip():
                count_rait += 1
                rating.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
                
        if len(temp_foundation) > 0:
            foundation.append(temp_foundation[0])
            temp_foundation = []
            
        if len(temp_count_n_stud) > 0:
            n_students.append(temp_count_n_stud[0])
            temp_count_n_stud = []
            
        break

    if count_f == 0 and count_age == 0 and count_l == 0 and count_n_stud == 0 and count_prog == 0 \
    and count_place == 0 and count_rait == 0 and count_adv == 0:
        for e in soup.find_all('div',class_ = 'accordion__box'):
            for e1 in e.find_all('p'):
                if 'основан' in e1.text.strip().lower() and ':' in e1.text.strip():
                    count_f += 1
                    temp_foundation.append(e1.text.strip().split(':')[-1].strip())
                if 'Возраст' in e1.text.strip() and (':' in e1.text.strip() or '–' in e1.text.strip()):
                    count_age += 1
                    age.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
                if ('Вид обучения' in e1.text.strip() 
                    or 'Формы обучения' in e1.text.strip()) and ':' in e1.text.strip():
                    count_l += 1
                    learning_type.append(e1.text.strip().split(':')[-1].strip())
                if 'количество' in e1.text.strip().lower() and ':' in e1.text.strip():
                    count_n_stud += 1
                    temp_count_n_stud.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
                if 'Программы' in e1.text.strip() and ':' in e1.text.strip():
                    count_prog += 1
                    programs.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
                if ('Размещение' in e1.text.strip() 
                    or 'Тип проживания' in e1.text.strip()) and ':' in e1.text.strip():
                    count_place += 1
                    placement.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
                if 'Рейтинг' in e1.text.strip() and ':' in e1.text.strip():
                    count_rait += 1
                    rating.append(unicodedata.normalize("NFKC", e1.text.strip().split(':')[-1].strip()))
                    
            if len(temp_foundation) > 0:
                foundation.append(temp_foundation[0])
                temp_foundation = []
                
            if len(temp_count_n_stud) > 0:
                n_students.append(temp_count_n_stud[0])
                temp_count_n_stud = []
                
            break

    # поиск дополнительных рейтинговых блоков
    if count_rait == 0:
        for e in soup.find_all('p'):
            if 'Рейтинг' in e.text and ':' in e.text:
                count_rait += 1
                rating.append(unicodedata.normalize("NFKC", e.find_next("ul").text).strip().replace('\n',' '))

    # поиск дополнительных блоков с программами
    if count_prog == 0:
        for e in soup.find_all('h2',class_ = 'accordion__title active'):        
            if 'Программы' in e.text:
                dop_prog=[]
                for e1 in e.find_all_next("span",itemprop="name"):
                    dop_prog.append(e1.text)
        dop_progs_text = ', '.join(dop_prog)
        count_prog += 1
        programs.append(dop_progs_text)
            
    # поиск блока Преимущества
    for e in soup.find_all('h2',class_ = 'accordion__title active'):        
        if 'Преимущества' in e.text:
            count_adv += 1
            adv = e.text.strip()
            advantage.append(unicodedata.normalize("NFKC", e.parent.text.strip().replace(adv,'').replace('\n',' ').strip()))

    if count_f == 0:
        foundation.append('Нет данных')
    if count_age == 0:
        age.append('Нет данных')    
    if count_l == 0:
        learning_type.append('Нет данных')
    if count_n_stud == 0:
        n_students.append('Нет данных') 
    if count_prog == 0:
        programs.append('Нет данных')                 
    if count_place == 0:
        placement.append('Нет данных')                     
    if count_rait == 0:
        rating.append('Нет данных')
    if count_adv == 0:
        advantage.append('Нет данных')  
    
    count+=1
    print(len(foundation),len(age),len(learning_type),len(n_students),len(programs),len(placement),len(rating),len(advantage),count)
    try:    
        assert (len(foundation)==len(age)==len(learning_type)==len(n_students)==len(programs)==len(placement)==len(rating)==len(advantage)==count),'Assert Error!'
    except:
        max_value = max(len(foundation),len(age),len(learning_type),len(n_students),len(programs),len(placement),len(rating),len(advantage))
        max_index = [len(foundation),len(age),len(learning_type),len(n_students),len(programs),len(placement),len(rating),len(advantage)].index(max_value)
        error[p_url] = max_index

        foundation = foundation[:count]
        age = age[:count]
        learning_type = learning_type[:count]
        n_students = n_students[:count]
        programs = programs[:count]
        placement = placement[:count]
        rating = rating[:count]
        advantage = advantage[:count]
    
    time.sleep(random.uniform(2, 3))

In [ ]:
df['Год основания'] = foundation
df['Возраст обучающихся'] = age
df['Вид обучения'] = learning_type
df['Количество учащихся'] = n_students
df['Программы обучения'] = programs
df['Размещение'] = placement
df['Рейтинг'] = rating
df['Преимущества'] = advantage

In [ ]:
with pd.ExcelWriter('_'.join(FQDN.split('/')[2].split('.')) + '.xlsx', 
                                engine_kwargs={'options': {'strings_to_urls': True}}) as writer:
    df.to_excel(writer, sheet_name='Лист1', index=False)
open_file('_'.join(FQDN.split('/')[2].split('.')) + '.xlsx')